In [1]:
import numpy as np
from scipy.stats import multivariate_normal
from scipy.special import logsumexp
import matplotlib.pyplot as plt

# **1) GENERATION DES DONNEES**

In [2]:
def generate_z(A,b,K,cov,x):
    z_O = []
    z_E = []
    for i in range(int(2**K)):
        z_O.append(np.random.multivariate_normal(A*x+b, (2/3)*cov))
        z_E.append(np.random.multivariate_normal(A*x+b, (2/3)*cov))
    z = z_O.copy()
    z.extend(z_E)
    return z, z_O, z_E

In [3]:
def generate_q_p(A,b,K,mean,cov,x,z,z_O,z_E):
    q = []
    p = []
    q_O = []
    p_O = []
    q_E = []
    p_E = []
    for i in range(int(2**(K+1))):
        q.append(multivariate_normal.pdf(z[i], mean=A*x+b, cov=(2/3)*cov))
        p.append(multivariate_normal.pdf(z[i], mean=mean, cov=cov)*multivariate_normal.pdf(x, mean=z[i], cov=cov))
    for i in range(int(2**(K))):
        q_O.append(multivariate_normal.pdf(z_O[i], mean=A*x+b, cov=(2/3)*cov))
        p_O.append(multivariate_normal.pdf(z_O[i], mean=mean, cov=cov)*multivariate_normal.pdf(x, mean=z_O[i], cov=cov))
        q_E.append(multivariate_normal.pdf(z_E[i], mean=A*x+b, cov=(2/3)*cov))
        p_E.append(multivariate_normal.pdf(z_E[i], mean=mean, cov=cov)*multivariate_normal.pdf(x, mean=z_E[i], cov=cov))
    return q, p, q_O, p_O, q_E, p_E

# **2) FONCTION POUR SS**

In [4]:
def SS_estimator(n_sim, mean, cov, A, b, r, theta):
    ss = []
    for i in range(n_sim):
        K = np.random.geometric(r, 1)
        x = np.random.multivariate_normal(np.zeros(20)+theta, 2*cov)
        z, z_O, z_E = generate_z(A,b,K,cov,x)
        q, p, q_O, p_O, q_E, p_E = generate_q_p(A,b,K,mean,cov,x,z,z_O,z_E)
        w = np.log(p) - np.log(q)
        w_O = np.log(p_O) - np.log(q_O)
        w_E = np.log(p_E) - np.log(q_E)
            
        #third step : compute I0
        I0 = w.mean()

        #fourth step : compute Lo and Le
            # Lo
        l_O = logsumexp(w_O) - np.log(len(w_O))    
            #Le
        l_E = logsumexp(w_E) - np.log(len(w_E))    

        #fifth step : compute L_O_E
        L_O_E = logsumexp(w) - np.log(len(w))
    
        #DeltaK
        DeltaK = L_O_E - 0.5*(l_O + l_E)
    
        #sixth step : compute the final estimator
        ml_ll_ss = I0 + (DeltaK/(r*(1-r)**(int(K)-1)))
        
        ss.append(ml_ll_ss)
    mean_value = np.mean(ss)
    std_error_value = np.std(ss) / np.sqrt(len(ss))

    return mean_value

#print("Mean: " + str(mean_value) + "\n" + "Standard error: " + str(std_error_value))

In [5]:
theta = np.random.normal(0,1)
SS_estimator(10000, np.zeros(20), np.identity(20), 1, 0, 0.6, theta)

-42.64213860481938

# **BOUCLE POUR ESTIMER THETA**

In [6]:
def estimate_theta(debut,fin,pas,n_sim):
    theta = np.random.normal(0,1)
    print(theta)
    theta_estim = []
    for i in range(n_sim):
        liste = np.linspace(debut,fin,pas)
        x = []
        y = []
        for k in liste:
            y.append(SS_estimator(100, np.zeros(20)+k, np.identity(20), 1, 0, 0.6,theta))
            x.append(k)
        theta_estim.append(x[y.index(max(y))])
        print(i) 
    return theta_estim, theta

In [7]:
#plt.scatter(x,y)
#estimate_theta(-3,3,90,3)

# **3) FONCTION POUR RUSSIAN ROULETTE**

In [8]:
def delta(k, q, p, q_O, p_O, q_E, p_E):
    w_O_k = []
    w_E_k = []
    w_k=[]
    for i in range(2**k):
        w_O_k.append(p_O[i]/q_O[i])
        w_E_k.append(p_E[i]/q_E[i])
    for i in range(2**(k+1)):
        w_k.append(p[i]/q[i])
    l_O_k = logsumexp(w_O_k) - np.log(len(w_O_k))  
    l_E_k = logsumexp(w_E_k) - np.log(len(w_E_k))    
    l_k = logsumexp(w_k) - np.log(len(w_k))
    delta_k = l_k - 0.5*(l_E_k + l_O_k)
    return delta_k

In [9]:
def RR_estimator(n_sim, mean, cov, A, b, r, theta):
    rr = []
    for i in range(n_sim):
        sum_rr = 0
        K = np.random.geometric(r, 1)
        x = np.random.multivariate_normal(mean, 2*cov)
        z, z_O, z_E = generate_z(A,b,K,cov,x)
        q, p, q_O, p_O, q_E, p_E = generate_q_p(A,b,K,mean,cov,x,z,z_O,z_E)
        w = np.log(p) - np.log(q)
        w_O = np.log(p_O) - np.log(q_O)
        w_E = np.log(p_E) - np.log(q_E)
            
        #third step : compute I0
        I0 = w.mean()
        
        for j in range(1,int(K)):
            sum_rr += (delta(j,q,p,q_O,p_O,q_E,p_E)/((1-r)**j))
            
        ml_ll_rr = I0 + sum_rr
        rr.append(ml_ll_rr)
        
    mean_value = np.mean(rr)
    std_error_value = np.std(rr) / np.sqrt(len(rr))

    return print("Mean: " + str(mean_value) + "\n" + "Standard error: " + str(std_error_value))

In [10]:
theta = np.random.normal(0,1)
RR_estimator(10000, np.zeros(20), np.identity(20), 1, 0, 0.6, theta)

Mean: -45.740345516551145
Standard error: 0.06747878839479356


# **4) FONCTION SUMO**

In [ ]:
def delta_sumo(k, q, p):
    w_k2=[]
    w_k1=[]
    for i in range(k+1):
        w_k1.append(p[i]/q[i])
    for i in range(k+2):
        w_k2.append(p[i]/q[i])
    l_k1 = np.log(np.mean(w_k1))
    l_k2 = np.log(np.mean(w_k2))
    delta_k = l_k2-l_k1
    return delta_k